In [2]:
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.word import WordTokenizer
from collections import defaultdict
import json
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import os
import re
import string
from cltk.tokenize.sentence import TokenizeSentence
from nltk.tokenize import sent_tokenize

In [9]:

def align_pars(lsentences,esentences):
    
    if lsentences[-1] == '':
        lsentences = lsentences[:-1]
    if esentences[-1] == '':
        esentences = esentences[:-1]
        
    if len(lsentences) > len(esentences):
        alignments = align(lsentences,esentences)
        target = esentences
        source = lsentences
    elif len(lsentences) < len(esentences):
        alignments = align(esentences,lsentences)
        target = lsentences
        source = esentences
    else:
        return zip(lsentences,esentences)
    
    sentence_tuples = []
    for k in alignments.keys():
        sentence_tuples.append((target[k], [source[j] for j in alignments[k]]))
        
    return sentence_tuples
          



def clean_lemma(word):
    try:
        word = re.sub('[0-9]+', '', word)
        word = re.sub('[-_]+', '', word)
    except:
        pass

    return word


def get_lemmata_defs(latin):
    words = lemmatizer.lemmatize(latin)
    ldefs = []
    unmatched = []

    for word in words:
        word = clean_lemma(word)
        try:
            ldefs.append(defs[word]['definition'].split(','))
        except:
            unmatched.append(word)
    return ldefs,unmatched


def get_defs(ldefs):
    lldefs=[]
    for l in ldefs:
        lldefs += t(l[0])
    return lldefs


def get_match_count(english,latin):
    
    ldefs,unmatched = get_lemmata_defs(latin)
    ldefs = get_defs(ldefs)
    count = 0
    for word in t(english):
        if word in string.punctuation:
            continue
        if word in ldefs and word not in stop:
            count += 1
        if word in unmatched:
            count += 1
    return count


def align(source,target):
    """ Takes a paragraph of sentences, source, and maps each sentence to a sentence in the 
        target paragraph. 
        
        Returns a dictionary with the target sentence indices as keys, and the sentences in the source paragraph
        which correspond to them as values.
    """
    tsize = len(target)
    ssize = len(source)
    alignments = defaultdict(list)
    alignments[0] = [0]
    i = 1
    j = 1
    while tsize < ssize and i < len(source) and j < tsize:
        max_count = get_match_count(source[i],target[j])
        max_i = j
        if j >= tsize-1:
            rge = [j-1]
        else:
            rge = [j-1,j+1]
        for r in rge: 
            c = get_match_count(source[i],target[r])
            if c > max_count:
                max_count = c
                max_i = r

        if max_i != j:
            ssize -= 1
        else:
            j += 1

        alignments[max_i].append(i)
        i += 1
        
  
    while j < tsize:
        alignments[j].append(i)
        j += 1
        i += 1
    alignments[j-1].extend([k for k in range(i,len(source))])
    
    return alignments

def get_sec_alignments(lpar,epar):
    if len(lpar) > len(epar):
        if len(epar) > 1:
            alignments = align(lpar,epar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(lpar[v])
                aligned.append((group,epar[k]))
        else:
            try:
                aligned = [('. '.join(lpar),epar[0])] 
            except:
                print('error, {}'.format(epar))
                aligned = list(zip(lpar,epar))
    elif len(epar) > len(lpar):
        if len(lpar) > 1:
            alignments = align(epar,lpar)
            aligned = []
            for k in alignments.keys():
                group = []
                for v in alignments[k]:
                    group.append(epar[v])
                aligned.append((lpar[k],group))
        else:
            try:
                aligned = [(lpar[0],'. '.join(epar))]
            except:
                print('error, {}'.format(lpar))
                aligned = list(zip(lpar,epar))
    else:
        aligned = list(zip(lpar,epar))
        
    return aligned

In [10]:
t = wordpunct_tokenize
stop = set(stopwords.words('english'))

f = open('../lemmas.json')
defs = json.loads(f.read())

lemmatizer = LemmaReplacer('latin')
tokenizer = WordTokenizer('latin')
s_tokenizer = TokenizeSentence('latin')

f = open('../pkls/tacitus_paragraphs.json')
paragraphs = json.loads(f.read())
f.close()


In [11]:
eng_text = []
lat_text = []

In [12]:
for p in paragraphs:
    eng_text.append(p[1])
    lat_text.append(p[0])

In [34]:
alignments = []

In [35]:
for c in range(len(eng_text)):
    es = ' '.join(eng_text[c].replace('\n',' ').split())
    ls = lat_text[c].replace('\n', ' ')

    ls = re.sub(r'\. \. \.','',ls)
    es = re.sub(r'\. \. \.','',es)
    ls = re.sub(r'[:;,]','',ls)
    es = re.sub(r'[:;,]','',es)
    ls = re.sub(r'(\d)+\.','',ls)
    es = re.sub(r'(\d)+\.','',es)

    
    epar = sent_tokenize(es)
    lpar = s_tokenizer.tokenize_sentences(ls)
    alignments.append(get_sec_alignments(lpar,epar))

In [37]:
count = 0
for a in alignments:
    count += len(a)

In [41]:
alignments[50]

[('  At Caesar cognita morte legati ne provincia sine rectore foret A. Didium suffecit.',
  [' The emperor on hearing of the death of his representative appointed Aulus Didius in his place that the province might not be left without a governor.']),
 ('is  propere vectus non tamen integras res invenit adversa interim legionis pugna cui Manlius Valens  praeerat auctaque et apud hostis eius rei fama quo venientem ducem exterrerrent atque illo augente  audita ut maior laus compositis et si duravissent venia iustior tribueretur.',
  ['Didius though he quickly arrived found matters far from prosperous for the legion under the command of Manlius Valens had meanwhile been defeated and the disaster had been exaggerated by the enemy to alarm the new general while he again magnified it that he might win the more glory by quelling the movement or have a fairer excuse if it lasted.']),
 ('Silures id quoque damnum  intulerant lateque persultabant donec adcursu Didii pellerentur.',
  ['This loss too 